# Table of Contents

* [References](#References)
* [Common Commands](#Common-Commands)
* [GDB peda Installation](#GDB-peda-Installation)
* [Examine Memory](#Examine-Memory)
* [Dumping Memory](#Dumping-Memory)

# References

1. [Cheat Sheet](http://darkdust.net/files/GDB%20Cheat%20Sheet.pdf)

# Common Commands

```
$ gdb <program>
 
$ break main
$ break *address
 
$ layout asm
$ layout regs
 
$ focus next | prev | src | asm | regs | split
# Set the focus to the named window. This command allows to change the
# active window so that scrolling keys can be affected to another window.
 
# to clear glitches in layout do
$ tui disable
$ tui enable
 
$ run
 
$ si # step instruction
$ ni # step over function call (assembly)
 
$ x/s $esi # print string esi is pointing to
$ x/20xg $rsp # examine stack layout
 
$ set $rip = 0x40060d
 
# demangles C++ function calls
$ set print asm-demangle on  #(default is off)
 
$ watch *[ADDRESS]
$ info breakpoints
$ info watchpoints
$ delete [WATCHPOINT_NUM | BREAKPOINT_NUM]
$ rwatch *0xfeedface # break on read
 
$ info functions [REGEX]
$ break [FUNCTIONNAME] 
# also tells you the address its at...i dont know a better way to find the function location :(
 
 
# running python commands
$ py print(0x60)
(gdb) py
>import sys
>print(sys.version)
>end
 
 
# Where you are on the stack
$ where
 
# calculator
$ p 4 * 4
$ p $r12 + $rbx * 8
 
 
# force disassembly (good for shell code)
$ x/10i $rip
```

# GDB peda Installation

The default version doesn't work with gdb compiled with Python3. I did find a [version](https://github.com/zachriggle/peda.git) that theoretically work with Python3 but had to change a lot of things and ended up not being worth it. 

Use the peda version from https://github.com/longld/peda.git to get a version that doesn't move the assembly code when you go to the next instruction.

This is worked for me:
```bash
sudo apt-get remove gdb
wget http://security.ubuntu.com/ubuntu/pool/main/g/gdb/gdb_7.4-2012.02-0ubuntu2_amd64.deb
sudo dpkg -i ./gdb_7.4-2012.02-0ubuntu2_amd64.deb
```

# Examine Memory

[Reference](http://www.delorie.com/gnu/docs/gdb/gdb_56.html)

x/nfu addr
+ `n`, the repeat count
+ `f`, the display format: `s` (null-terminated string), or `i` (machine instruction). The default is `x` (hexadecimal) initially. The default changes each time you use either `x` or print.
+ `u`, the unit size: `b` – bytes, `h` – halfwords (2 bytes), `w` – words (4 bytes), `g` – giantwords (8 bytes)

# Dumping Memory

```
# This will give you a plain binary dump int file result.bin.
$ dump binary memory result.bin 0x200000000 0x20000c350
$ dump binary memory comparison_array.bin $esi-28 ($esi-28+0x1000)
 
# Easiest to turn into array
$ dump verilog memory comparison_array.bin $esi-28 ($esi-28+0x1000)
```

# Changing code in memory
[Reference](http://mohit.io/blog/debugging-modifying-code-at-runtime/)
```
gdb-peda$ x/10i $rip
=> 0x555555554b9e <_Z6decodePhi+116>:	cmp    eax,0xf
   0x555555554ba1 <_Z6decodePhi+119>:	jle    0x555555554bc7 <_Z6decodePhi+157>
gdb-peda$ x/10b 0x555555554ba1
0x555555554ba1 <_Z6decodePhi+119>:	0x7d	0x24
gdb-peda$ set *(char*)0x555555554ba1=0x7d
gdb-peda$ x/10i $rip
=> 0x555555554b9e <_Z6decodePhi+116>:	cmp    eax,0xf
   0x555555554ba1 <_Z6decodePhi+119>:	jge    0x555555554bc7 <_Z6decodePhi+157>
```